In [1]:
!pip install peft transformers datasets emoji==0.6.0
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=fa22cd933c4f9c2bced0247c177550b402a7988fd28a7cce568c16a6575d4b45
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [2]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from peft import LoraConfig,get_peft_model
import json
import torch

In [ ]:
# import tensorflow as tf
# print(tf.__version__)
# !pip3 install --upgrade tensorflow

2.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstal

In [3]:
from transformers import pipeline, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification,DataCollatorWithPadding
train_data_file_path = "/content/NHL-SentiComments-5K-TRAIN.json"
test_data_file_path = "/content/NHL-SentiComments-1K-TEST.json"

In [ ]:
with open(train_data_file_path, 'r', encoding = 'utf-8') as file:
  train_data = json.load(file)
with open(test_data_file_path, 'r', encoding = 'utf-8') as file:
  test_data = json.load(file)
label_to_int = {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
}
def convert_to_dict(data):
  new_data = []
  for entry in data:
    new_elm = {}
    new_elm["text"] = entry["comment"]
    new_elm["label"] = label_to_int[entry["label"]]
    new_data.append(new_elm)
  return new_data

train_data = convert_to_dict(train_data)
eval_data = convert_to_dict(test_data)

# Convert the data into a format suitable for training
train_dataset = Dataset.from_dict({"text": [item["text"] for item in train_data], "label": [item["label"] for item in train_data]})
eval_dataset = Dataset.from_dict({"text": [item["text"] for item in eval_data], "label": [item["label"] for item in eval_data]})

# Define the sentiment analysis model and tokenizer
# model_name ='cardiffnlp/twitter-roberta-base-sentiment-latest'  # You can choose a different model
model_name = 'finiteautomata/bertweet-base-sentiment-analysis'  # You can choose a different model
tokenizer = AutoTokenizer.from_pretrained(model_name)


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: positive, negative, neutral


# Tokenize the input data
def tokenize_batch(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize_batch, batched=True)
eval_dataset = eval_dataset.map(tokenize_batch, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer = tokenizer);

peft_config = LoraConfig(
    task_type = "SEQ_CLS",
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0.01,
)

model = get_peft_model(model,peft_config)

# # Define the training arguments
training_args = TrainingArguments(
    output_dir="./sentiment_results",
    gradient_accumulation_steps=8,
    per_device_train_batch_size=4,
    warmup_steps=50,
    max_steps=5000,
    learning_rate=3.0e-4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    overwrite_output_dir=True,
    num_train_epochs=2,
    load_best_model_at_end=True
    # ,weight_decay=0.01
)

# # Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# # Fine-tune the sentiment analysis model
trainer.train()

# # Save the fine-tuned model
model.save_pretrained("./sentiment_fine_tuned_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Map:   0%|          | 0/5168 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
0,No log,0.562538
2,No log,0.558750
4,0.611300,0.546238
6,0.456000,0.626606
8,0.456000,0.700786
10,0.323200,0.731110
12,0.233300,0.796343
14,0.233300,0.976566
16,0.172400,1.079761


In [ ]:
# Create a pipeline using the local model and tokenizer
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer,truncation=True)

successes = 0
for i in range(0,len(eval_data)):

  if (i % 50 == 0 ):
    print(i)
  # print(i)
  data = eval_data[i]
  our_label = data['label']
  model_label = classifier(data['text'])
  model_label = label_to_int[model_label[0]['label']]
  if model_label == our_label:
    successes+=1

accurary_score = successes / len(eval_data)

In [ ]:
print(accurary_score)

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Access evaluation metrics
eval_loss = eval_results['eval_loss']
print("Evaluation Loss:", eval_loss)

# New Section

In [ ]:
import shutil

shutil.rmtree("/content/sentiment_results/")

In [ ]:
from transformers import pipeline, RobertaTokenizer, RobertaForSequenceClassification

# Load local model and tokenizer
model_path = "/content/sentiment_fine_tuned_model"

model = RobertaForSequenceClassification.from_pretrained(model_path)

# Create a pipeline using the local model and tokenizer
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer,truncation=True)

successes = 0
for i in range(0,len(eval_data)):
  print(i)
  data = eval_data[i]
  our_label = data['label']
  model_label = classifier(data['text'])
  model_label = label_to_int[model_label[0]['label']]
  if model_label == our_label:
    successes+=1

accurary_score = successes / len(eval_data)

In [ ]:
print(accurary_score)

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Access evaluation metrics
eval_loss = eval_results['eval_loss']
print("Evaluation Loss:", eval_loss)
